# Đồ án môn Lập trình song song ứng dụng
**Giảng viên: Thầy Trần Trung Kiên**

## Thông tin nhóm
### STT: Nhóm 1

### Thành viên:
1. 1712892 - Nguyễn Thị Tuyết
2. 1712284 - Hoàng Gia Bảo
3. 1712685 - Trần Huy Phượng

## 1. Mô tả ứng dụng

Tên ứng dụng: **Phát hiện cộng đồng trong mạng xã hội bằng thuật toán Girvan Newman**

* Input: Đồ thị vô hướng, không trọng số

* Output: Tập hợp các cộng đồng được phát hiện - mỗi cộng đồng được thể hiện là list các node

* Ý nghĩa thực tế: có rất nhiều, ví dụ một số ứng dụng:
  * Tiếp thị: xác định các cụm khách hàng có chung sở thích trong một mạng thể hiện quan hệ giữa người mua và sản phẩm trên sàn thương mại điện tử để xây dựng hệ thống tư vấn hiệu quả
  * Kết bạn, mở rộng mối quan hệ
  * ...


Ứng dụng này cần phải được tăng tốc do vì khi recommend 1 sản phẩm cho khách hàng, ta cần thời gian nhanh để đem lại trải nghiệm tốt nhất cho khách hàng. Trong khi đó thuật toán Girvan Newman có thời gian thực thi khá chậm nên việc song song hóa là rất cần thiết.

## 2. Tiền xử lí dữ liệu

### 1. Tải dữ liệu

In [1]:
!wget https://raw.githubusercontent.com/mansiganatra/Girvan-Newman-Implementation-using-Spark/master/ub_sample_data.csv

--2022-05-03 05:52:25--  https://raw.githubusercontent.com/mansiganatra/Girvan-Newman-Implementation-using-Spark/master/ub_sample_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1777828 (1.7M) [text/plain]
Saving to: ‘ub_sample_data.csv.5’

ub_sample_data.csv. 100%[===================>]   1.70M  --.-KB/s    in 0.01s   

2022-05-03 05:52:25 (164 MB/s) - ‘ub_sample_data.csv.5’ saved [1777828/1777828]



In [2]:
import csv

In [3]:
fileData = []
header = []
with open('ub_sample_data.csv', newline ='') as csvfile:
    file = csv.reader(csvfile, delimiter=',')
    for row in file:
        if not header:
            header.append(row)
            continue
        fileData.append(row)
    csvfile.close()

### 2. Tạo graph từ input

#### Tạo graph

In [4]:
filterThreshold = 7
setUsers = {}

for [user, item] in fileData:
    if user not in setUsers:
        setUsers[user] = {item}
    else:
        setUsers[user].add(item)

gPyObject = {}

# filter and make adjacency list of graph
for user1, item1 in setUsers.items():
    for user2, item2 in setUsers.items():
        if user1 == user2: continue
        if(len(item1 & item2) >= filterThreshold):
            if user1 not in gPyObject:
                gPyObject[user1] = [user2]
            else:
                gPyObject[user1].append(user2)
cnt = 0
for key, val in gPyObject.items():
    print(key, val)
    cnt += 1
    if cnt == 5:
        break

39FT2Ui8KUXwmUt6hnwy-g ['0FVcoJko1kfZCrJRfssfIA', 'JM0GL6Dx4EuZ1mprLk5Gyg', 'bSUS0YcvS7UelmHvCzNWBA', 'DKolrsBSwMTpTJL22dqJRQ', 'sdLns7062kz3Ur_b8wgeYw', '_VTEyUzzH92X3w-IpGaXVA', 'qtOCfMTrozmUSHWIcohc6Q', 'zBi_JWB5uUdVuz3JLoAxGQ', 'KLB3wIYUwKDPMbijIE92vg', '1KQi8Ymatd4ySAd4fhSfaw', '_Pn-EmWO-pFPFg81ZIEiDw', 'dTeSvET2SR5LDF_J07wJAQ', 'ay4M5J28kBUf0odOQct0BA', 'dzJDCQ5vubQBJTfYTEmcbg', 'mu4XvWvJOb3XpG1C_CHCWA', 'OoyQYSeYNyRVOmdO3tsxYA', 'PE8s8ACYABRNANI-T_WmzA', '79yaBDbLASfIdB-C2c8DzA', 'Uo5dPwoDpYBzOnmUnjxJ6A', 'R4l3ONHzGBakKKNo4TN9iQ', 'bHufZ2OTlC-OUxBDRXxViw', 'ChshgCKJTdIDg17JKtFuJw', 'B0ENvYKQdNNr1Izd2r-BAA']
0FVcoJko1kfZCrJRfssfIA ['39FT2Ui8KUXwmUt6hnwy-g', 'LcCRMIDz1JgshpPGYfLDcA', 'bSUS0YcvS7UelmHvCzNWBA', 'DKolrsBSwMTpTJL22dqJRQ', '_VTEyUzzH92X3w-IpGaXVA', 'qtOCfMTrozmUSHWIcohc6Q', 'zBi_JWB5uUdVuz3JLoAxGQ', 'EiwxlbR8fb68lMgEXhcWKA', '4pc_EyanaC3ARh0MZZyouA', 'KLB3wIYUwKDPMbijIE92vg', '2XYdguaaZ7dgi6fAlddujg', 'bE7Yd0jI_P6g27MWEKKalA', '1KQi8Ymatd4ySAd4fhSfaw', '_Pn-EmWO-pFPFg8

#### Thay đổi định dạng user từ string sang integer

In [5]:
transferUser = {}
cnt = 0
for key in gPyObject.keys():
    transferUser[key] = cnt
    cnt += 1
g = [[] for _ in range(cnt)]
for key, val in gPyObject.items():
    user1 = transferUser[key]
    for it in val:
        user2 = transferUser[it]
        g[user1].append(user2)
g[:5]

[[1,
  3,
  5,
  9,
  11,
  12,
  14,
  15,
  18,
  22,
  23,
  24,
  26,
  29,
  31,
  35,
  41,
  61,
  67,
  68,
  86,
  88,
  93],
 [0,
  4,
  5,
  9,
  12,
  14,
  15,
  16,
  17,
  18,
  19,
  21,
  22,
  23,
  24,
  26,
  27,
  35,
  42,
  44,
  45,
  46,
  49,
  53,
  54,
  62,
  71,
  73,
  93,
  94,
  95,
  96,
  97,
  104,
  108,
  110,
  117,
  118,
  120,
  123,
  128,
  129,
  130,
  133,
  136,
  139,
  140,
  146,
  155,
  156,
  159,
  175,
  177],
 [6, 85],
 [0,
  9,
  10,
  13,
  18,
  20,
  25,
  29,
  31,
  32,
  33,
  35,
  36,
  38,
  39,
  58,
  63,
  66,
  78,
  81,
  89,
  90,
  98,
  106,
  112,
  115,
  145,
  148],
 [1, 12, 15, 17, 44, 48, 56, 62, 121]]

#### Ghi graph ra file

In [6]:
with open('graph.txt', "w+") as fp:
    fp.write(str(cnt))
    fp.write("\n")
    for i in g:
        line = " ".join([str(item) for item in i])
        fp.write(line)
        fp.write("\n")

    fp.close()

## 3. Cài đặt tuần tự

### 1. Thiết kế

Các bước thực hiện:
1. Xác định độ đo betweenness cho các cạnh trong đồ thị bằng thuật toán Girvan Newman
2. Loại bỏ cạnh có betweenness lớn nhất khỏi graph
3. Tính độ đo modularity để lưu lại cách chia communities tốt nhất
4. Thực hiện lại bước 2 đến khi không còn cạnh nào

https://colab.research.google.com/drive/1R2Gt2wPEeEL-9_kGfXJRUy24zv1APglX?usp=sharing

### 2. Đánh giá

#### Kịch bản thí nghiệm

Thực hiện 2 version: \
    + ver1: code python bình thường và thêm decorator @jit \
    + ver2: chuyển code về sử dụng mảng numpy và sử dụng decorator @jit(nopython=True)

#### Chạy code và xem kết quả

In [7]:
import numba
import numpy as np

##### Version 1

In [8]:
# Load graph format for our code
f = open("graph.txt", "r")
vertices = int(f.readline())
g = []
for i in range(vertices):
    line = list(map(int, f.readline().split()))
    g.append(line)
f.close()

In [9]:
from queue import Queue

In [10]:
# Run code
bet = [[0 for i in range(vertices)] for i in range(vertices)]
INF = 1e9

In [11]:
def bfs(x : int):
    visited = [False for i in range(vertices)] # marked visited
    level = [INF for i in range(vertices)] # level of each vertices
    parents = [0 for i in range(vertices)] # number of parent lv = lv-1
    point = [1 for i in range(vertices)] # point of each vertices
    q = Queue() # method: put/ get/ empty
    q.put(x)
    level[x], parents[x] = 0, 1
    maxlv = 0
    while not q.empty():
        u = q.get()
        visited[u] = True
        for v in g[u]:
            if level[v] + 1 == level[u]: # meet your parent vertices
                parents[u] += 1
                continue
            if not visited[v]: # meet new vertices
                visited[v] = True
                level[v] = level[u] + 1
                maxlv = max(maxlv, level[v])
                q.put(v)
    
    lv = [[] for i in range(maxlv + 1)]
    for i in range(vertices):
        idx = int(level[i])
        if idx != INF:
            lv[idx].append(i)

    for lvleaf in range(maxlv, 0, -1):
        for leaf in lv[lvleaf]:
            for parent in g[leaf]:                
                if level[parent] + 1 == lvleaf:
                    bet[parent][leaf] += point[leaf] / parents[leaf]
                    bet[leaf][parent] += point[leaf] / parents[leaf]
                    point[parent] += point[leaf] / parents[leaf]

In [12]:
normalizationFactor = vertices ** 2 - vertices + 1 # C=(n-1)^{2}-(n-1)
resBetv1 = []

In [13]:
def betweenness():
    for i in range(vertices):
      bfs(i)
    for i in range(vertices):
        for j in range(vertices):
            if j < i: continue
            if bet[i][j]: 
                bet[i][j] *= (2 / (vertices * (vertices - 1)))
                bet[i][j] /= 2
                bet[j][i] = bet[i][j]
                resBetv1.append(f"({i}, {j}): {bet[i][j]:.6f}")

In [14]:
%%time
betweenness()

CPU times: user 442 ms, sys: 4.93 ms, total: 447 ms
Wall time: 608 ms


In [15]:
resBetv1

['(0, 1): 0.024585',
 '(0, 3): 0.055945',
 '(0, 5): 0.006611',
 '(0, 9): 0.014234',
 '(0, 11): 0.007705',
 '(0, 12): 0.005960',
 '(0, 14): 0.003409',
 '(0, 15): 0.005200',
 '(0, 18): 0.004045',
 '(0, 22): 0.003672',
 '(0, 23): 0.001974',
 '(0, 24): 0.002114',
 '(0, 26): 0.006145',
 '(0, 29): 0.007364',
 '(0, 31): 0.003213',
 '(0, 35): 0.000929',
 '(0, 41): 0.007705',
 '(0, 61): 0.007705',
 '(0, 67): 0.022869',
 '(0, 68): 0.004777',
 '(0, 86): 0.003998',
 '(0, 88): 0.007705',
 '(0, 93): 0.004250',
 '(1, 4): 0.015619',
 '(1, 5): 0.004281',
 '(1, 9): 0.088824',
 '(1, 12): 0.003333',
 '(1, 14): 0.002093',
 '(1, 15): 0.001386',
 '(1, 16): 0.011333',
 '(1, 17): 0.001651',
 '(1, 18): 0.020074',
 '(1, 19): 0.004476',
 '(1, 21): 0.002947',
 '(1, 22): 0.002566',
 '(1, 23): 0.001688',
 '(1, 24): 0.001283',
 '(1, 26): 0.019754',
 '(1, 27): 0.002681',
 '(1, 35): 0.013886',
 '(1, 42): 0.004947',
 '(1, 44): 0.001588',
 '(1, 45): 0.001737',
 '(1, 46): 0.007603',
 '(1, 49): 0.002343',
 '(1, 53): 0.0044

##### Version 2

In [16]:
# Load graph
f = open("graph.txt", "r")
vertices = int(f.readline())
g_pyObj = []
# print(vertices)
for i in range(vertices):
    g_pyObj.append(list(map(int, f.readline().split())))
f.close()

sizeOfG = np.zeros(vertices, dtype=int)
g = np.zeros((vertices, vertices), dtype=int)
for i in range(vertices):
    for j in range(len(g_pyObj[i])):
        g[i][j] = g_pyObj[i][j]
    sizeOfG[i] = len(g_pyObj[i])

bet = np.zeros((vertices, vertices), dtype=float)

In [17]:
# @numba.jit(nopython=True)
def bfs_v2(x : int, g : np.array, bet : np.array):
    visited = np.zeros(vertices, dtype=int)
    level = np.empty(vertices, dtype=int)
    parents = np.zeros(vertices, dtype=float)
    point = np.empty(vertices, dtype=float)
    point.fill(1)
    level.fill(INF)
    q = np.zeros(vertices, dtype=int) # put/ get/ empty
    l, r = 0, 0 # left right is begin and end of queue
    q[r] = x; r += 1 # put operator
    level[x], parents[x] = 0, 1; maxlv = 0; visited[x] = 1
    while l < r:
        u = q[l]
        l += 1       # pop operator
        for vindex in range(sizeOfG[u]):
            v = g[u][vindex]
            if level[v] + 1 == level[u]: # meet your parent vertices
                parents[u] += 1
                continue
            if not visited[v]: # meet new vertices
                visited[v] = 1
                level[v] = level[u] + 1
                if maxlv < level[v]:
                    maxlv = level[v]
                q[r] = v; r += 1 # put operator

    # lv = [[] for i in range(maxlv + 1)]
    lv = np.empty((maxlv + 1, vertices), dtype=int)
    sizeOfLv = np.zeros(maxlv + 1, dtype=int)
    for i in range(vertices):
        if level[i] != INF:
            newPosition = sizeOfLv[level[i]]
            lv[level[i]][newPosition] = i
            sizeOfLv[level[i]] += 1

    # print(lv)
    # print(sizeOfLv)
    # print(parents)

    for leafLevel in range(maxlv, 0, -1):
        for leafIndex in range(sizeOfLv[leafLevel]): #lv[lvleaf]:
            # leaf vertices if lv[leaf]
            leaf = lv[leafLevel][leafIndex]

            for connectedNodeIndex in range(sizeOfG[leaf]):
                node = g[leaf][connectedNodeIndex]                
                if level[node] + 1 == leafLevel:
                    if parents[leaf] == 0: continue
                    bet[node][leaf] += point[leaf] / parents[leaf]
                    bet[leaf][node] += point[leaf] / parents[leaf]
                    point[node] += point[leaf] / parents[leaf]

In [18]:
normalizationFactor = vertices ** 2 - vertices + 1 # C=(n-1)^{2}-(n-1)
resBetv2 = []

In [19]:
def betweenness_v2():
    for i in range(vertices):
        bfs_v2(i, g, bet)
    for i in range(vertices):
        for j in range(vertices):
            if j < i: continue
            if bet[i][j]: 
                bet[i][j] *= (2 / (vertices * (vertices - 1)))
                bet[i][j] /= 2
                bet[j][i] = bet[i][j]
                resBetv2.append(f"({i}, {j}): {bet[i][j]:0.6f}")

In [20]:
%%time
betweenness_v2()

CPU times: user 1.02 s, sys: 4.78 ms, total: 1.02 s
Wall time: 1.42 s


In [21]:
resBetv2

['(0, 1): 0.024585',
 '(0, 3): 0.055945',
 '(0, 5): 0.006611',
 '(0, 9): 0.014234',
 '(0, 11): 0.007705',
 '(0, 12): 0.005960',
 '(0, 14): 0.003409',
 '(0, 15): 0.005200',
 '(0, 18): 0.004045',
 '(0, 22): 0.003672',
 '(0, 23): 0.001974',
 '(0, 24): 0.002114',
 '(0, 26): 0.006145',
 '(0, 29): 0.007364',
 '(0, 31): 0.003213',
 '(0, 35): 0.000929',
 '(0, 41): 0.007705',
 '(0, 61): 0.007705',
 '(0, 67): 0.022869',
 '(0, 68): 0.004777',
 '(0, 86): 0.003998',
 '(0, 88): 0.007705',
 '(0, 93): 0.004250',
 '(1, 4): 0.015619',
 '(1, 5): 0.004281',
 '(1, 9): 0.088824',
 '(1, 12): 0.003333',
 '(1, 14): 0.002093',
 '(1, 15): 0.001386',
 '(1, 16): 0.011333',
 '(1, 17): 0.001651',
 '(1, 18): 0.020074',
 '(1, 19): 0.004476',
 '(1, 21): 0.002947',
 '(1, 22): 0.002566',
 '(1, 23): 0.001688',
 '(1, 24): 0.001283',
 '(1, 26): 0.019754',
 '(1, 27): 0.002681',
 '(1, 35): 0.013886',
 '(1, 42): 0.004947',
 '(1, 44): 0.001588',
 '(1, 45): 0.001737',
 '(1, 46): 0.007603',
 '(1, 49): 0.002343',
 '(1, 53): 0.0044

#### Kiểm tra độ chính xác


Kiểm tra độ chính xác bằng thư viện NetworkX

In [22]:
import networkx as nx
import matplotlib.pyplot as plt

%matplotlib inline

In [23]:
# Load graph format for NetworkX
f = open("graph.txt", "r")
vertices = int(f.readline())
nx_g = []
# print(vertices)
for i in range(vertices):
    line = list(map(int, f.readline().split()))
    for val in line:
        # print(val)
        nx_g.append((i, val))
f.close()

In [24]:
G = nx.Graph(nx_g)
nx_bet = nx.edge_betweenness_centrality(G, normalized=True)

res_nx_bet = []
for node in sorted(nx_bet):
  res_nx_bet.append(f"({node[0]}, {node[1]}): {nx_bet[node]:.6f}")

res_nx_bet

['(0, 1): 0.026011',
 '(0, 3): 0.057107',
 '(0, 5): 0.006944',
 '(0, 9): 0.015812',
 '(0, 11): 0.007705',
 '(0, 12): 0.006352',
 '(0, 14): 0.003315',
 '(0, 15): 0.005288',
 '(0, 18): 0.004088',
 '(0, 22): 0.003673',
 '(0, 23): 0.002081',
 '(0, 24): 0.002182',
 '(0, 26): 0.006230',
 '(0, 29): 0.007227',
 '(0, 31): 0.003248',
 '(0, 35): 0.000935',
 '(0, 41): 0.007705',
 '(0, 61): 0.007705',
 '(0, 67): 0.022869',
 '(0, 68): 0.004875',
 '(0, 86): 0.003999',
 '(0, 88): 0.007705',
 '(0, 93): 0.004356',
 '(1, 4): 0.015926',
 '(1, 5): 0.004289',
 '(1, 9): 0.090360',
 '(1, 12): 0.003337',
 '(1, 14): 0.002125',
 '(1, 15): 0.001417',
 '(1, 16): 0.011333',
 '(1, 17): 0.001659',
 '(1, 18): 0.021595',
 '(1, 19): 0.004503',
 '(1, 21): 0.002956',
 '(1, 22): 0.002566',
 '(1, 23): 0.001710',
 '(1, 24): 0.001286',
 '(1, 26): 0.016604',
 '(1, 27): 0.002686',
 '(1, 35): 0.013476',
 '(1, 42): 0.004932',
 '(1, 44): 0.001661',
 '(1, 45): 0.001746',
 '(1, 46): 0.007603',
 '(1, 49): 0.002338',
 '(1, 53): 0.0045

In [25]:
# TESTING
print(resBetv1 == resBetv2)

True


In [26]:
err = 0.0
for node in nx_bet:
  print(nx_bet[node], bet[node[0]][node[1]])
  if bet[node[0]][node[1]] == 205.0: # diff here
    print(node[0], node[1])
    break
  err += abs(nx_bet[node] - bet[node[0]][node[1]])
  # break
print(err)

0.026010663935131566 0.02458475160681044
0.0571065761374769 0.05594535833506412
0.006943691135127072 0.006611311942194291
0.015811981988452527 0.014233577468871612
0.007704537116301822 0.007704537116301822
0.00635159001909169 0.0059599960335254415
0.0033148533148533213 0.003409376571141277
0.005288153211475066 0.005200092424357135
0.004087830558418794 0.00404451801510625
0.0036729036729036737 0.0036722242604595658
0.0020812829636359105 0.0019743361655126406
0.002181630195449787 0.0021135075363016512
0.006230212112565054 0.006145164233399548
0.007226803739979141 0.0073637005622299734
0.0032482708953297185 0.0032131962279021097
0.0009352112293288768 0.0009290965173318121
0.007704537116301822 0.007704537116301822
0.007704537116301822 0.007704537116301822
0.022869022869022867 0.022869022869022867
0.004874784286548981 0.004777118747706983
0.00399902164608047 0.003998342233636366
0.007704537116301822 0.007704537116301822
0.004355519061401403 0.004249724837960132
0.01592630551906565 0.0156193

## Cài đặt song song

### 1. Thiết kế

### 2. Đánh giá

#### Kịch bản thí nghiệm

#### Chạy code và xem kết quả

#### Kiểm tra độ chính xác

## Cài đặt song song & tối ưu hóa

### 1. Thiết kế

### 2. Đánh giá

#### Kịch bản thí nghiệm

#### Chạy code và xem kết quả

#### Kiểm tra độ chính xác